In [1]:
import pandas as pd
from unidecode import unidecode
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/zeus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('../base/review.csv',encoding='latin-1')

In [3]:
df.head()

,SentimentText,Sentiment
0,"first think another Disney movie, might good, ...",1
1,"Put aside Dr. House repeat missed, Desperate H...",0
2,"big fan Stephen King's work, film made even gr...",1
3,watched horrid thing TV. Needless say one movi...,0
4,truly enjoyed film. acting terrific plot. Jeff...,1


In [4]:
import string
from nltk.stem.snowball import SnowballStemmer
import swifter
import nltk

stemmer = SnowballStemmer("english")
stop = set(stopwords.words('english'))
def lower(texto):
    return texto.lower()

def normalize(texto):
    return unidecode(texto)

def remove_ponctuation(texto):
    for punc in string.punctuation:
        texto = texto.replace(punc," ")
    return texto

def remove_stopwords(texto):
    ret = []
    for palavra in texto.split():
        if palavra not in stop:
            ret.append(palavra)
    return ' '.join(ret)

def stem(texto):

    ret = []
    for palavra in texto.split():
        ret.append(stemmer.stem(palavra))
    return ' '.join(ret)


def remove_number(texto):
    result = ''.join([i for i in texto if not i.isdigit()])
    return result

def pipeline(texto):
    texto = normalize(texto)
    texto = lower(texto)
    texto = remove_ponctuation(texto)
    texto = remove_stopwords(texto)
    texto = remove_number(texto)
    texto = stem(texto)
    return texto

In [5]:
df['SentimentText'].apply(lower).head()

0    first think another disney movie, might good, ...
1    put aside dr. house repeat missed, desperate h...
2    big fan stephen king's work, film made even gr...
3    watched horrid thing tv. needless say one movi...
4    truly enjoyed film. acting terrific plot. jeff...
Name: SentimentText, dtype: object

In [6]:
remove_ponctuation("é, ué!")

'é  ué '

In [7]:
len(df)

25000

In [8]:
df['preproc'] = df['SentimentText'].swifter.apply(pipeline)

In [9]:
# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(df['preproc'])
# len(vectorizer.get_feature_names())

In [10]:
vectorizer_tfidf = TfidfVectorizer()
X = vectorizer_tfidf.fit_transform(df['preproc'])
len(vectorizer_tfidf.get_feature_names())

49824

In [11]:
y = df['Sentiment']

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
import numpy as np

clf = LogisticRegression(solver='liblinear')
np.mean(cross_val_score(clf,X, y, cv=10,scoring='balanced_accuracy'))

0.88784

In [13]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
np.mean(cross_val_score(clf,X, y, cv=10,scoring='balanced_accuracy'))

0.8633599999999999

In [14]:
clf.fit(X,y)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [15]:
import pickle
filename = 'clf.pickle'
outfile = open(filename,'wb')
pickle.dump(clf,outfile)
outfile.close()

In [16]:
filename = 'vectorizer.pickle'
outfile = open(filename,'wb')
pickle.dump(vectorizer_tfidf,outfile)
outfile.close()

In [17]:
#I just love this movie. Specially the climax, seriously one of the best climax I have ever seen.

In [18]:
#I just want to say how amazing this film is from start to finish. This will take you on a emotional ride.You will not he disappointed

In [19]:
#LITERALLY , one of the best movies i have seen in my entire life , filled with a tone of action and emotions . you will love avenger endgame . ' i love you 3000 '